In [ ]:
"""
!pip uninstall -y numpy
!pip install numpy==1.26.4
import os
os.kill(os.getpid(), 9)

import numpy as np
import torch
print("PyTorch:", torch.__version__)
print("NumPy:", np.__version__)
"""

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 117.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
!pip install numpy==1.26.4
!pip install torch==2.2.0 torchvision==0.17.0 --index-url https://download.pytorch.org/whl/cu121
!pip install diffusers==0.31.0 transformers==4.30.0 peft==0.5.0 einops safetensors>=0.3.1 albumentations matplotlib kornia

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.3/757.3 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 122.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 104.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 127.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/1

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

# Create the models-checkpoints directory if it doesn't exist.
os.makedirs("models-checkpoints", exist_ok=True)

# Install gdown if it's not already installed.
!pip install --upgrade gdown

# Download the Unet Latent Edge Predictor checkpoint.
# This command downloads the checkpoint into the models-checkpoints directory with the filename "unet_latent_edge_predictor_checkpoint.pt"
!gdown https://drive.google.com/uc?id=1w7eimErXnnRrjY6TY8yXrmno-hvcZ94_ -O models-checkpoints/unet_latent_edge_predictor_checkpoint.pt

# Download the Sketch Simplification Network checkpoint.
# Replace <URL_SKETCH> with the actual URL for model_gan.pth.
!cp "/content/drive/MyDrive/models-checkpoints/model_gan.pth" "./models-checkpoints/model_gan.pth"

print("Environment setup complete. Checkpoints downloaded to 'models-checkpoints'.")

Downloading...
From (original): https://drive.google.com/uc?id=1w7eimErXnnRrjY6TY8yXrmno-hvcZ94_
From (redirected): https://drive.google.com/uc?id=1w7eimErXnnRrjY6TY8yXrmno-hvcZ94_&confirm=t&uuid=f7331ae7-d27e-4f6c-8539-14292b9f0781
To: /content/models-checkpoints/unet_latent_edge_predictor_checkpoint.pt
100% 437M/437M [00:03<00:00, 120MB/s]
Environment setup complete. Checkpoints downloaded to 'models-checkpoints'.


In [4]:
# Standard imports
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import torch
import torch.nn as nn
from torch.utils.data import random_split
import numpy
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
# Hugging Face Hub import

# Diffusers-specific imports
from diffusers import StableDiffusionPipeline, DDIMScheduler
from peft import get_peft_model, LoraConfig, PeftModel

# Custom modules

from models import UNETLatentEdgePredictor, SketchSimplificationNetwork
from pipeline import SketchGuidedText2Image

from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import random
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm



In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Load Sketch Simplifier

In [6]:
# Configure and load sketch simplification network

sketch_simplifier = SketchSimplificationNetwork().to(device)
sketch_simplifier.load_state_dict(torch.load("models-checkpoints/model_gan.pth"))

sketch_simplifier.eval()
sketch_simplifier.requires_grad_(False)

SketchSimplificationNetwork(
  (0): Conv2d(1, 48, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (1): ReLU()
  (2): Conv2d(48, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU()
  (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): ReLU()
  (6): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (7): ReLU()
  (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU()
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU()
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (13): ReLU()
  (14): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU()
  (16): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU()
  (18): Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (19): ReLU()
  (20): Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1

# Load Stable Diffusian Model and schdueler for Infernce

In [ ]:
from dotenv import load_dotenv
import os

#!huggingface-cli login
# Load Stable Diffusion Pipeline
token =  "Your token"# Replace with your actual token
pipeline = StableDiffusionPipeline.from_pretrained("benjamin-paine/stable-diffusion-v1-5", use_auth_token=token)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `GoogleKey` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticat

model_index.json:   0%|          | 0.00/543 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Keyword arguments {'use_auth_token': 'REMOVED_TOKEN'} are not expected by StableDiffusionPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
/usr/local/lib/python3.11/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [9]:
# Load Stable Diffusion Pipeline
stable_diffusion_1_5 = "benjamin-paine/stable-diffusion-v1-5"
stable_diffusion=StableDiffusionPipeline.from_pretrained(
    stable_diffusion_1_5,
    torch_dtype=torch.float16,
    safety_checker=None  # Skip the safety checker if it's not required
)
vae = stable_diffusion.vae.to(device)
unet = stable_diffusion.unet.to(device)
tokenizer = stable_diffusion.tokenizer
text_encoder = stable_diffusion.text_encoder.to(device)

vae.eval()
unet.eval()
text_encoder.eval()

text_encoder.requires_grad_(False)

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


CLIPTextModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 768)
      (position_embedding): Embedding(77, 768)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e

# Unet Pipeline and model

In [10]:
# Load U-Net latent edge predictor
checkpoint = torch.load("models-checkpoints/unet_latent_edge_predictor_checkpoint.pt", map_location=torch.device('cpu'))
LEP_UNET = UNETLatentEdgePredictor(9320, 4, 9).to(device)
LEP_UNET.load_state_dict(checkpoint["model_state_dict"])
LEP_UNET.eval()
LEP_UNET.requires_grad_(False)

UNETLatentEdgePredictor(
  (e1): encoder_block(
    (conv): convolutional_block(
      (conv1): Conv2d(9320, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (e2): encoder_block(
    (conv): convolutional_block(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (

# Apply Lora Finetuning

In [11]:
# Apply LoRA to the U-Net
lora_config = LoraConfig(
    r=16,  # Rank of LoRA matrix
    lora_alpha=32,  # Scaling factor
    target_modules=[
        "conv_out",  # Fine-tune final UNet output
        "up_blocks.2.attentions.0.transformer_blocks.0.attn1.to_q",
        "up_blocks.2.attentions.0.transformer_blocks.0.attn1.to_k",
        "up_blocks.2.attentions.0.transformer_blocks.0.attn1.to_v"
    ],
    lora_dropout=0.1
)

unet = get_peft_model(unet, lora_config)

In [12]:

for name, param in unet.named_parameters():
    if param.requires_grad:
        print(f"LoRA is fine-tuning: {name} | Shape: {param.shape}")



LoRA is fine-tuning: base_model.model.up_blocks.2.attentions.0.transformer_blocks.0.attn1.to_q.lora_A.default.weight | Shape: torch.Size([16, 640])
LoRA is fine-tuning: base_model.model.up_blocks.2.attentions.0.transformer_blocks.0.attn1.to_q.lora_B.default.weight | Shape: torch.Size([640, 16])
LoRA is fine-tuning: base_model.model.up_blocks.2.attentions.0.transformer_blocks.0.attn1.to_k.lora_A.default.weight | Shape: torch.Size([16, 640])
LoRA is fine-tuning: base_model.model.up_blocks.2.attentions.0.transformer_blocks.0.attn1.to_k.lora_B.default.weight | Shape: torch.Size([640, 16])
LoRA is fine-tuning: base_model.model.up_blocks.2.attentions.0.transformer_blocks.0.attn1.to_v.lora_A.default.weight | Shape: torch.Size([16, 640])
LoRA is fine-tuning: base_model.model.up_blocks.2.attentions.0.transformer_blocks.0.attn1.to_v.lora_B.default.weight | Shape: torch.Size([640, 16])
LoRA is fine-tuning: base_model.model.conv_out.lora_A.default.weight | Shape: torch.Size([16, 320, 3, 3])
LoRA i

In [13]:
for name, param in unet.named_parameters():
    if "lora" not in name:  # Only allow LoRA layers to update
        param.requires_grad = False

# Setting up dataset


In [14]:
import os
from torch.utils.data import Dataset
from PIL import Image

class SketchImageDataset(Dataset):
    def __init__(self, root_dir, mapping_file=None, image_transform=None):
        self.root_dir = root_dir
        self.image_dir = os.path.join(root_dir, "photos")
        self.sketch_dir = os.path.join(root_dir, "sketch")
        # Collect all image filenames with common image extensions.
        self.image_filenames = [f for f in os.listdir(self.image_dir) if f.endswith((".jpg", ".png"))]
        self.image_transform = image_transform  # transform for photos only

        # Load mapping from file if provided.
        # Expect each line in the mapping file to be formatted as "key: value"
        self.mapping = {}
        print(os.path.exists(mapping_file))
        if mapping_file is not None and os.path.exists(mapping_file):
            with open(mapping_file, 'r') as f:
                for line in f:
                    line = line.strip()
                    if not line:
                        continue  # Skip empty lines.
                    if ":" in line:
                        key, value = line.split(":", 1)
                        self.mapping[key.strip()] = value.strip()

    def clean_filename(self, filename):
        """
        Removes file extension and extra characters.
        """
        return os.path.splitext(filename)[0]

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):


        # Load the photo.
        image_name = self.image_filenames[idx]
        image_path = os.path.join(self.image_dir, image_name)
        image = Image.open(image_path).convert("RGB")
        if self.image_transform:
            image = self.image_transform(image)

        base_name = self.clean_filename(image_name)
        # Assume each photo has a corresponding folder named "base_name.jpg" in the sketch directory.
        sketch_folder = os.path.join(self.sketch_dir, base_name + ".jpg")
        if not os.path.exists(sketch_folder):
            raise FileNotFoundError(f"Sketch folder not found: {sketch_folder}")

        # Instead of selecting a single sketch, load all sketches that contain "sketchs32strokes"
        sketch_files = [f for f in os.listdir(sketch_folder)
                        if f.endswith((".jpg", ".png")) and ("sketchs32strokes" in f or "sketchs20strokes" in f)]
        if not sketch_files:
            raise ValueError(f"No sketches with 'sketchs32strokes' found for {image_name}")
        # Load all matching sketches into a list of PIL images.
        sketches = [Image.open(os.path.join(sketch_folder, f)).convert("RGB") for f in sketch_files]

        # Construct the text prompt.
        text_prompt = "Lego " + self.mapping.get(base_name, base_name) + " with a white background"

        return {"image": image, "sketches": sketches, "text_prompt": text_prompt, "filename": image_name}




In [15]:

from torch.utils.data import random_split, DataLoader
# Initialize dataset and dataloader
image=512
photo_transform = transforms.Compose([
    transforms.Resize((image,image)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])


def custom_collate_fn(batch):
    images = torch.stack([item["image"] for item in batch])  # Assuming images are tensors.
    text_prompts = [item["text_prompt"] for item in batch]
    filenames = [item["filename"] for item in batch]
    # sketches remains as a list of lists of PIL images.
    sketches = [item["sketches"] for item in batch]
    return {"image": images, "sketch": sketches, "text_prompt": text_prompts, "filename": filenames}


# Initialize the dataset.
dataset = SketchImageDataset(
    root_dir="/content/drive/MyDrive/Lego_256x256/Clipasso",
    mapping_file="mappings.txt",
    image_transform=photo_transform
)


dataset_size = len(dataset)

train_size = int(0.8 * dataset_size)
val_size = int(0.1 * dataset_size)
test_size = dataset_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=custom_collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False, collate_fn=custom_collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=custom_collate_fn)
print(f"📊 Dataset split sizes: Train={train_size}, Validation={val_size}, Test={test_size}")



True
📊 Dataset split sizes: Train=32, Validation=4, Test=5


# Noise/ Optimizer


In [16]:
for name, param in unet.named_parameters():
    if "lora" not in name:
        param.requires_grad = False

In [17]:
import numpy as np
import torch
print(torch.__version__)
print(np.__version__)

2.2.0+cu121
1.26.4


In [18]:
# Noise Scheduler
noise_scheduler = DDIMScheduler(
        beta_start = 0.00085,
        beta_end = 0.012,
        beta_schedule = "scaled_linear",
        num_train_timesteps = 1000,
        clip_sample = False,
    )


# Optimizer
# Define parameter groups with separate learning rates
param_groups = [
    {'params': [], 'lr': 1e-3},  # higher LR for conv_out
    {'params': [], 'lr': 5e-4}   # lower LR for attention layers
]

# Assign parameters to groups explicitly
for name, param in unet.named_parameters():
    if param.requires_grad:
        if "conv_out" in name:
            param_groups[0]['params'].append(param)
        else:  # all other LoRA layers
            param_groups[1]['params'].append(param)

# Setup AdamW with separate parameter groups
optimizer = torch.optim.AdamW(
    param_groups,
    weight_decay=0.01,
    eps=1e-8
)

# Pipeline Setup

In [19]:
import importlib
import pipeline
import TrainingPipelineTest
importlib.reload(TrainingPipelineTest)
from TrainingPipelineTest import SketchGuidedText2ImageTrainer  # Import again


# Initialize trainer
trainer = SketchGuidedText2ImageTrainer(
    stable_diffusion_pipeline=stable_diffusion,
    unet=unet,
    vae=vae,
    text_encoder=text_encoder,
    lep_unet=LEP_UNET,
    scheduler=noise_scheduler,
    tokenizer=tokenizer,
    sketch_simplifier=sketch_simplifier,
    device=device
)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 190MB/s]


#  Training

In [21]:
import csv
accumulation_steps = 10  # effective batch size becomes 4
optimizer.zero_grad()
num_inference_timesteps=20
num_epochs = 18
seed=1000
runtime=f"Test{accumulation_steps}epoch{num_epochs}andLoraMidblocks"

# CSV Files
training_log_csv = os.path.join(runtime, "training_log.csv")
image_loss_csv = os.path.join(runtime, "image_loss.csv")
epoch_log_csv = os.path.join(runtime, "epoch_avg_loss.csv")
gpu_usage_csv = os.path.join(runtime, "gpu_usage.csv")
detail_loss_csv = f"{runtime}/detail_loss.csv"
weightscsv = os.path.join(runtime, "weightchange.csv")

# Create directories
os.makedirs(f"{runtime}", exist_ok=True)

# Initialize CSV files
with open(training_log_csv, "w", newline="") as f:
    csv.writer(f).writerow(["Epoch", "Iteration", "Accumulated Loss", "Average Loss", "Avg VGG Loss", "Avg SSIM", "Avg Edge"])

with open(epoch_log_csv, "w", newline="") as f:
    csv.writer(f).writerow(["Epoch", "Avg MSE", "Avg VGG", "Avg SSIM", "Avg Edge"])

with open(gpu_usage_csv, "w", newline="") as f:
    csv.writer(f).writerow(["Epoch", "Iteration", "Avg GPU Usage (MB)"])

with open(image_loss_csv, "w", newline="") as f:
    csv.writer(f).writerow(["Epoch", "Image Filename", "MSE", "Scaled Loss", "VGG", "White", "SSIM", "Edge"])

# Initialize CSV logging
with open(detail_loss_csv, "w", newline="") as f:
    csv.writer(f).writerow(["Epoch", "MSE Loss", "VGG Loss", "White Loss", "Total Loss", "SSIM", "Edge"])

# Initialize weight logging
with open(weightscsv, "w", newline="") as f:
    csv.writer(f).writerow(["Epoch","VGG weight"])


In [22]:
best_loss = float('inf')
best_val_loss = float('inf')
best_runtime = None
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
ssimw, edgew, msew, vggw = 0.5, 0.5, 1.0, 0.5
prev_ssim_loss, prev_edge_loss, prev_mse_loss, prev_vgg_loss = 0, 0, 0, 0
os.makedirs(f'{runtime}/output_images', exist_ok=True)
for epoch in range(num_epochs):
    accumulated_loss,accumulated_vgg_loss,accumulated_ssim_loss,accumulated_edge_loss, total_epoch_loss = 0.0, 0.0, 0.0, 0.0,0.0
    gpu_usage_accumulated, accumulation_counter = 0.0, 0
    total_epoch_ssim_loss, total_epoch_edge_loss, total_epoch_vgg_loss = 0.0, 0.0, 0.0
    unet.train()

    for i, batch in enumerate(train_dataloader):
        images, sketches, text_prompts, filenames = batch["image"].to(device), batch["sketch"], batch["text_prompt"], batch["filename"]
        loss, guidance_loss, gpu_usage, ssim_loss, edge_loss, white_loss, vgg = trainer.train_step(
            images, sketches, text_prompts, optimizer, noise_scheduler,
            num_inference_timesteps=num_inference_timesteps,
            classifier_guidance_strength=6,
            sketch_guidance_strength=1.6,
            guidance_steps_perc=0.6,
            seed=seed,
        )

        total_loss = (loss * msew + vggw*vgg
        #+0.1*white_loss
                      )
        batch_loss = total_loss / accumulation_steps
        batch_loss.backward()

        # Accumulate loss statistics
        accumulated_loss += loss.item()
        accumulated_vgg_loss += vgg.item()
        accumulated_ssim_loss += ssim_loss.item()
        accumulated_edge_loss += edge_loss.item()


        total_epoch_loss += loss.item()
        total_epoch_vgg_loss += vgg.item()
        total_epoch_ssim_loss += ssim_loss.item()
        total_epoch_edge_loss += edge_loss.item()
        gpu_usage_accumulated += torch.cuda.max_memory_allocated(device) / (1024**2)

        # Log per-image losses
        with open(image_loss_csv, "a", newline="") as f:
            for filename in filenames:
                csv.writer(f).writerow([
                    epoch + 1, filename,
                    loss.item(), batch_loss.item(),
                    vgg.item(), white_loss.item(),
                    ssim_loss.item(), edge_loss.item()
                ])


        accumulation_counter+= 1
        # Perform optimizer step after accumulation steps
        if (i + 1) % accumulation_steps == 0 or accumulation_counter == accumulation_steps:
            avg_loss = accumulated_loss / accumulation_steps
            avg_vgg_loss = accumulated_vgg_loss / accumulation_steps
            avg_ssim_loss = accumulated_ssim_loss / accumulation_steps
            avg_edge_loss = accumulated_edge_loss / accumulation_steps
            avg_gpu_usage = gpu_usage_accumulated / accumulation_steps

            torch.nn.utils.clip_grad_norm_(unet.parameters(), 1.0)
            # Dynamic weight adjustment
            """
            if prev_ssim_loss is not None and prev_edge_loss is not None:
                if avg_ssim_loss >= prev_ssim_loss:
                    ssimw *= 1.05
                elif avg_ssim_loss < prev_ssim_loss * 0.9:
                    ssimw *= 0.95

                if avg_ssim_loss < prev_ssim_loss and avg_edge_loss < prev_edge_loss:
                    edgew *= 1.05
                elif avg_edge_loss > prev_edge_loss * 1.1:
                    edgew *= 0.95

                if avg_loss < prev_mse_loss * 0.95:
                    msew *= 0.95

                if avg_vgg_loss >= prev_vgg_loss:
                    vggw *= 1.05
                elif avg_vgg_loss < prev_vgg_loss * 0.9:
                    vggw *= 0.95

            prev_ssim_loss, prev_edge_loss, prev_mse_loss, prev_vgg_loss = avg_ssim_loss, avg_edge_loss, avg_loss, avg_vgg_loss
            print(f"epoch {epoch+1} ssimw: {ssimw}, edgew{edgew}, mse_weight{msew}")


            with open(weightscsv, "a", newline="") as f:
                csv.writer(f).writerow([epoch + 1,ssimw, edgew, msew])

            """
            # Write accumulated losses to training_log.csv
            with open(training_log_csv, "a", newline="") as f:
                csv.writer(f).writerow([
                    epoch + 1, i + 1,
                    accumulated_loss, avg_loss,
                    avg_vgg_loss, avg_ssim_loss, avg_edge_loss
                ])


            # Log average GPU usage
            with open(gpu_usage_csv, "a", newline="") as f:
                csv.writer(f).writerow([epoch + 1, i + 1, avg_gpu_usage])

            # Reset accumulated values
            accumulated_loss = accumulated_vgg = accumulated_white = 0.0
            accumulated_ssim = accumulated_edge = 0.0
            gpu_usage_accumulated = 0.0

            print(f"Epoch {epoch+1} Iteration {i+1} | Avg Loss: {avg_loss:.4f} | Avg VGG: {avg_vgg_loss} |SSIM: {avg_ssim_loss:.4f} | Edge Loss: {avg_edge_loss:.4f} | GPU Usage: {avg_gpu_usage:.2f} MB")

            optimizer.step()
            optimizer.zero_grad()

    # Compute final epoch averages
    epoch_avg_train_loss = total_epoch_loss / len(train_dataloader)
    epoch_avg_ssim_loss = total_epoch_ssim_loss / len(train_dataloader)
    epoch_avg_edge_loss = total_epoch_edge_loss / len(train_dataloader)
    epoch_avg_vgg_loss = total_epoch_vgg_loss / len(train_dataloader)
    with open(epoch_log_csv, "a", newline="") as f:
        csv.writer(f).writerow([epoch + 1, epoch_avg_train_loss, epoch_avg_vgg_loss, epoch_avg_ssim_loss, epoch_avg_edge_loss])

    print(f"Epoch {epoch+1} | Training Avg Loss: {epoch_avg_train_loss:.4f} | vgg {epoch_avg_vgg_loss}  |avr ssim {epoch_avg_ssim_loss:.4f} | avrg edge {epoch_avg_edge_loss:.4f}")

    # Validation phase
    total_val_mse = 0.0
    total_val_vgg = 0.0
    total_val_white = 0.0
    total_val_ssim = 0.0
    total_val_edge = 0.0
    total_samples = 0
    unet.eval()

    with torch.no_grad():  # No gradients computed for validation
        val_mse, val_ssim, val_edge_loss = 0, 0, 0
        total_epoch_val_loss = 0.0
        total_epoch_guidance_loss = 0.0
        total_samples = 0
        for batch in val_dataloader:
          images, sketches, text_prompts,filenames = (
                batch["image"].to(device),
                batch["sketch"],
                batch["text_prompt"],
                batch["filename"]
            )
          # Iterate over each image, its sketches, and text prompt
          for img, sketches, prompt, filename in zip(images, sketches, text_prompts, filenames):
              img = img.unsqueeze(0)  # Add batch dimension for single image
              losses_per_sketch = []
              guidance_losses_per_sketch = []
              mse_list, vgg_list, white_list, ssim_list, edge_list = [], [], [], [], []
              iteration=0
              # Evaluate all sketches individually
              for sketch in sketches:
                  val_loss, guidance_loss, ssim_loss, edge_loss, whiteloss, vgg, generated_image, edgemap  = trainer.eval(
                      images=img,
                      sketches=[sketch],  # Pass single sketch as a list
                      text_prompts=[prompt],
                      optimizer=None,  # No optimizer step during eval
                      noise_scheduler=noise_scheduler,
                      num_inference_timesteps=num_inference_timesteps,
                      classifier_guidance_strength=8,
                      sketch_guidance_strength=1.8,
                      guidance_steps_perc=0.5,
                      seed=None
                  )
                  mse_list.append(val_loss.item())
                  vgg_list.append(vgg.item())
                  white_list.append(whiteloss.item())
                  ssim_list.append(ssim_loss.item())
                  edge_list.append(edge_loss.item())
                  generated_image.save(f'{runtime}/output_images/sketch: {iteration} name: {filename}.png')
                  iteration=iteration+1


              # Average losses per image (across sketches)
              avg_mse_img = sum(mse_list) / len(mse_list)
              avg_vgg_img = sum(vgg_list) / len(vgg_list)
              avg_white_img = sum(white_list) / len(white_list)
              avg_ssim_img = sum(ssim_list) / len(ssim_list)
              avg_edge_img = sum(edge_list) / len(edge_list)

              # Correct accumulation here:
              total_val_mse += avg_mse_img
              total_val_vgg += avg_vgg_img
              total_val_white += avg_white_img
              total_val_ssim += avg_ssim_img
              total_val_edge += avg_edge_img

              total_samples += 1

        # Correctly compute epoch averages:
        avg_val_mse = total_val_mse / total_samples
        avg_val_vgg = total_val_vgg / total_samples
        avg_val_white = total_val_white / total_samples
        avg_val_ssim = total_val_ssim / total_samples
        avg_val_edge = total_val_edge / total_samples
        # Compute total validation loss with dynamic weighting
        validation_total_loss = (
            avg_val_mse*msew +
            vggw * avg_val_vgg
            #+0.1 * avg_val_white
        )

        #print(f"Epoch {epoch+1} | Total Validation Loss: {validation_total_loss:.4f} | MSE: {epoch_avg_val_loss:.4f} | SSIM: {epoch_avg_ssim_loss:.4f} | Edge Loss: {epoch_avg_edge_loss:.4f}")
        print(f"Epoch {epoch+1} | Total Validation Loss: {validation_total_loss:.4f} | "
          f"MSE: {avg_val_mse:.4f} |VGG Loss: {avg_val_vgg:.4f} | SSIM: {avg_val_ssim:.4f} | "
          f"Edge Loss: {avg_val_edge:.4f} ")

        # Log validation detail loss
        with open(detail_loss_csv, "a", newline="") as f:
            csv.writer(f).writerow([
                epoch + 1,
                avg_val_mse, avg_val_vgg, avg_val_white,
                validation_total_loss, avg_val_ssim, avg_val_edge
            ])

    """
    # Adjust weights dynamically
    if prev_ssim_loss is not None and prev_edge_loss is not None and prev_vgg_loss is not None:
        # Adjust SSIM weight
        if epoch_avg_ssim_loss >= prev_ssim_loss:
            ssimw *= 1.05
        elif epoch_avg_ssim_loss < prev_ssim_loss * 0.9:
            ssimw *= 0.95

        # Adjust Edge weight
        if epoch_avg_ssim_loss < prev_ssim_loss and epoch_avg_edge_loss < prev_edge_loss:
            edgew *= 1.05
        elif epoch_avg_edge_loss > prev_edge_loss * 1.1:
            edgew *= 0.95

        # Adjust VGG weight
        if epoch_avg_vgg_loss >= prev_vgg_loss:
            vggw *= 1.05
        elif epoch_avg_vgg_loss < prev_vgg_loss * 0.9:
            vggw *= 0.95

    prev_ssim_loss, prev_edge_loss, prev_vgg_loss = epoch_avg_ssim_loss, epoch_avg_edge_loss, epoch_avg_vgg_loss

    # Log weight changes
    with open(weightscsv, "a", newline="") as f:
        csv.writer(f).writerow([epoch + 1, ssimw, edgew, msew, vggw])
    """

    # Choose best model based on validation loss only
    if validation_total_loss  < best_val_loss:
        best_val_loss = validation_total_loss
        best_runtime = epoch+1
        unet.save_pretrained(f"{runtime}/best_unet_epoch{epoch+1}")
        print(f"✅ New Best Model Saved at Epoch {epoch+1} (Total Loss: {validation_total_loss:.4f})")
    else:
        unet.save_pretrained(f"{runtime}/unet_epoch{epoch+1}")

    if epoch % 5 == 0:  # every 5 epochs
      if epoch_avg_vgg_loss > prev_vgg_loss:
          vggw *= 1.03  # slight increase
      elif epoch_avg_vgg_loss < prev_vgg_loss * 0.95:
          vggw *= 0.97  # slight decrease
      with open(weightscsv, "a", newline="") as f:
        csv.writer(f).writerow([epoch + 1, vggw])
      prev_vgg_loss = epoch_avg_vgg_loss

    scheduler.step(validation_total_loss)


FileNotFoundError: Sketch folder not found: /content/drive/MyDrive/Lego_256x256/Clipasso/sketch/Pirate_Ship.jpg

1: Steps=20, Cls=6, Sketch=1.6, Perc=0.5, Image Cosine=0.909, Edge Cosine=0.909, Avg Cosine=0.909

2: Steps=20, Cls=6, Sketch=1.6, Perc=0.6, Image Cosine=0.898, Edge Cosine=0.898, Avg Cosine=0.898

3: Steps=20, Cls=6, Sketch=1.8, Perc=0.5, Image Cosine=0.895, Edge Cosine=0.895, Avg Cosine=0.895

4: Steps=20, Cls=4, Sketch=1.8, Perc=0.5, Image Cosine=0.892, Edge Cosine=0.892, Avg Cosine=0.892

5: Steps=20, Cls=4, Sketch=1.0, Perc=0.6, Image Cosine=0.888, Edge Cosine=0.888, Avg Cosine=0.888

6: Steps=20, Cls=4, Sketch=1.8, Perc=0.4, Image Cosine=0.888, Edge Cosine=0.888, Avg Cosine=0.888

7: Steps=20, Cls=4, Sketch=1.0, Perc=0.7, Image Cosine=0.885, Edge Cosine=0.885, Avg Cosine=0.885

8: Steps=20, Cls=6, Sketch=1.0, Perc=0.7, Image Cosine=0.883, Edge Cosine=0.883, Avg Cosine=0.883

9: Steps=20, Cls=6, Sketch=1.6, Perc=0.7, Image Cosine=0.880, Edge Cosine=0.880, Avg Cosine=0.880

10: Steps=20, Cls=6, Sketch=1.0, Perc=0.6, Image Cosine=0.879, Edge Cosine=0.879, Avg Cosine=0.879


# Save model


#Download Folder

In [ ]:
!zip -r 3.4MultipleSketchaccum10epoch18.zip 3.4MultipleSketchaccum10epoch18

In [ ]:
from google.colab import files
files.download("3.4MultipleSketchaccum10epoch18.zip")

In [ ]:
#unet.save_pretrained(f"{runtime}/best_unet_epoch{epoch+1}")
from peft import PeftModel

stable_diffusion=StableDiffusionPipeline.from_pretrained(
    stable_diffusion_1_5,
    torch_dtype=torch.float16,
    safety_checker=None
)
vae = stable_diffusion.vae.to(device)
unet = stable_diffusion.unet.to(device)
tokenizer = stable_diffusion.tokenizer
text_encoder = stable_diffusion.text_encoder.to(device)

#Load LoRA adapters
unet = PeftModel.from_pretrained(unet, f"./3.4MultipleSketchaccum10epoch18/best_unet_epoch{best_runtime}")


# Evaluate  Model


In [ ]:
import importlib
import pipeline
import TrainingPipelineTest
importlib.reload(TrainingPipelineTest)
from TrainingPipelineTest import SketchGuidedText2ImageTrainer  # Import again


# Initialize trainer
eval = SketchGuidedText2ImageTrainer(
    stable_diffusion_pipeline=stable_diffusion,
    unet=unet,
    vae=vae,
    text_encoder=text_encoder,
    lep_unet=LEP_UNET,
    scheduler=noise_scheduler,
    tokenizer=tokenizer,
    sketch_simplifier=sketch_simplifier,
    device=device
)

In [ ]:
import torch
import csv

# Evaluation CSV Logging
runtime = runtime
os.makedirs(f"/content/{runtime}", exist_ok=True)

# Validation phase
total_val_mse = 0.0
total_val_vgg = 0.0
total_val_white = 0.0
total_val_ssim = 0.0
total_val_edge = 0.0
total_samples = 0
unet.eval()

with torch.no_grad():  # No gradients computed for validation
  val_mse, val_ssim, val_edge_loss = 0, 0, 0
  total_epoch_val_loss = 0.0
  total_epoch_guidance_loss = 0.0
  total_samples = 0
  for batch in val_dataloader:
    images, sketches, text_prompts,filenames = (
          batch["image"].to(device),
          batch["sketch"],
          batch["text_prompt"],
          batch["filename"]
      )
    # Iterate over each image, its sketches, and text prompt
    for img, sketches, prompt, filename in zip(images, sketches, text_prompts, filenames):
        img = img.unsqueeze(0)  # Add batch dimension for single image
        losses_per_sketch = []
        guidance_losses_per_sketch = []
        mse_list, vgg_list, white_list, ssim_list, edge_list = [], [], [], [], []
        iteration=0
        # Evaluate all sketches individually
        for sketch in sketches:
            val_loss, guidance_loss, ssim_loss, edge_loss, whiteloss, vgg, generated_image, edgemap  = trainer.eval(
                images=img,
                sketches=[sketch],  # Pass single sketch as a list
                text_prompts=[prompt],
                optimizer=None,  # No optimizer step during eval
                noise_scheduler=noise_scheduler,
                num_inference_timesteps=num_inference_timesteps,
                classifier_guidance_strength=8,
                sketch_guidance_strength=1.8,
                guidance_steps_perc=0.5,
                seed=None
            )
            mse_list.append(val_loss.item())
            vgg_list.append(vgg.item())
            white_list.append(whiteloss.item())
            ssim_list.append(ssim_loss.item())
            edge_list.append(edge_loss.item())
            generated_image.save(f'output_images/sketch: {iteration} name: {filename}.png')
            sketches=sketches+1


        # Average losses per image (across sketches)
        avg_mse_img = sum(mse_list) / len(mse_list)
        avg_vgg_img = sum(vgg_list) / len(vgg_list)
        avg_white_img = sum(white_list) / len(white_list)
        avg_ssim_img = sum(ssim_list) / len(ssim_list)
        avg_edge_img = sum(edge_list) / len(edge_list)

        # Correct accumulation here:
        total_val_mse += avg_mse_img
        total_val_vgg += avg_vgg_img
        total_val_white += avg_white_img
        total_val_ssim += avg_ssim_img
        total_val_edge += avg_edge_img

        total_samples += 1

    # Correctly compute epoch averages:
    avg_val_mse = total_val_mse / total_samples
    avg_val_vgg = total_val_vgg / total_samples
    avg_val_white = total_val_white / total_samples
    avg_val_ssim = total_val_ssim / total_samples
    avg_val_edge = total_val_edge / total_samples
    # Compute total validation loss with dynamic weighting
    validation_total_loss = (
        avg_val_mse*msew +
        vggw * avg_val_vgg+ 0.1 * avg_val_white
    )

    #print(f"Epoch {epoch+1} | Total Validation Loss: {validation_total_loss:.4f} | MSE: {epoch_avg_val_loss:.4f} | SSIM: {epoch_avg_ssim_loss:.4f} | Edge Loss: {epoch_avg_edge_loss:.4f}")
    print(f"Epoch {epoch+1} | Total Validation Loss: {validation_total_loss:.4f} | "
    f"MSE: {avg_val_mse:.4f} |VGG Loss: {avg_val_vgg:.4f} | SSIM: {avg_val_ssim:.4f} | "
    f"Edge Loss: {avg_val_edge:.4f} ")


# Log epoch summary clearly
with open(f"/content/{runtime}/epoch_eval_summary.csv", "w", newline='') as epoch_summary_file:
    summary_writer = csv.writer(epoch_summary_file)
    summary_writer.writerow(["Epoch", "MSE Loss", "VGG Loss", "White Loss", "Total Loss", "SSIM", "Edge"])
    summary_writer.writerow([1,avg_val_mse, avg_val_vgg, avg_val_white,
            validation_total_loss, avg_val_ssim, avg_val_edge
        ])


print(f"🎯 Testing completed ")



In [ ]:
!zip -r 3.4MultipleSketchaccum10epoch18_2.zip 3.4MultipleSketchaccum10epoch18

In [ ]:
from google.colab import files
files.download("3.4MultipleSketchaccum10epoch18_2.zip")

In [ ]:
from google.colab import runtime
runtime.unassign()